In [18]:
from instagram import Account, Media, WebAgent #взять отсюда https://github.com/OlegYurchik/pyInstagram
import torch
import torchvision
import torchvision.models as models
import torch.nn as nn
from torchvision import transforms

In [42]:
MAX_COUNT = 20 #сколько максимальо фоток мы хотим загрузить
accountName = "mama_na_kuxne" #название аккаунта

agent = WebAgent()
account = Account(accountName)

link_list = []
pointer = None

media, pointer = agent.get_media(account, count = MAX_COUNT) 
for med in media:
    try:
        link_list.append(med.resources[0])
    except Exception:
        continue


In [43]:
img_dimensions = 224

img_transforms = transforms.Compose([
    transforms.Resize((img_dimensions, img_dimensions)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225] )
    ])

resnet_loaded = torch.hub.load('pytorch/vision', 'resnet18')
resnet_loaded.fc = nn.Sequential(nn.Linear(resnet_loaded.fc.in_features,512),nn.ReLU(), nn.Dropout(), nn.Linear(512, 2))
resnet_loaded.load_state_dict(torch.load('./models/resnet18.pth', map_location=torch.device('cpu')))
resnet_loaded.eval()

if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

Using cache found in C:\Users\flash/.cache\torch\hub\pytorch_vision_master


In [44]:
import requests
from PIL import Image
from io import BytesIO

img_list = [] #здесь будут храниться PIL-image фотографий еды
food_link_list = []
for link in link_list:
    try:
        response = requests.get(link, stream=True)
        img = Image.open(BytesIO(response.content))
        img = img_transforms(img)
        img = img.unsqueeze(0)
        resnet_loaded.to(device)
        prediction = resnet_loaded(img.to(device))
        prediction = prediction.argmax()
        if (prediction == 1):
            img_list.append(img)
            food_link_list.append(link)
            print(link)
        del response
    except Exception:
        continue

https://instagram.fhel4-1.fna.fbcdn.net/v/t51.2885-15/e35/s150x150/123228945_3329629723802191_5889572144787479607_n.jpg?_nc_ht=instagram.fhel4-1.fna.fbcdn.net&_nc_cat=107&_nc_ohc=YTu7hzrhbqIAX8MNUAe&tp=15&oh=e59dbeae8445b6e39d3d15caef11f78b&oe=5FB682CE
https://instagram.fhel4-1.fna.fbcdn.net/v/t51.2885-15/e35/c0.180.1440.1440a/s150x150/122828130_1004110713391430_4407864791860020883_n.jpg?_nc_ht=instagram.fhel4-1.fna.fbcdn.net&_nc_cat=102&_nc_ohc=_zYGolUh2JIAX97OJei&tp=16&oh=9736c8cf8809559542a3688ffbf1eb79&oe=5FDE8B60
https://instagram.fhel4-1.fna.fbcdn.net/v/t51.2885-15/e35/p150x150/122724098_682436929051109_9081446399525587438_n.jpg?_nc_ht=instagram.fhel4-1.fna.fbcdn.net&_nc_cat=105&_nc_ohc=XsaOxaSbl0wAX8CTj3s&tp=19&oh=87e3bf41bb6f15142da63b1d131efd9a&oe=5FB64F02
https://instagram.fhel4-1.fna.fbcdn.net/v/t51.2885-15/e35/s150x150/118201582_154040039654066_1311475974897409879_n.jpg?_nc_ht=instagram.fhel4-1.fna.fbcdn.net&_nc_cat=102&_nc_ohc=gA0B5Kp9PFoAX9DPTwi&tp=15&oh=7a56e069fc94ac984